 ## Introduction to Julia

**Spencer Lyon, NYU Stern Economics**

*Date: 8-11-15*

### What is Julia?


> Julia is a high-level, high-performance dynamic programming language for technical computing...
>
> -- <cite>[julialang.org](http://julialang.org)</cite>

- How many have heard of Julia?
- How many have used Julia?

### Key features

- Multiple dispatch: the core Julia abstraction $\Longrightarrow$ specialize function behavior based on runtime types of **all** arguments (classic OO specializes on type of *first* object only)

- Built in package manager $\Longrightarrow$ easy and officially supported package installation, even with binary dependencies

- JIT compiled (think numba) $\Longrightarrow$ concise high level (python-esque) code that can run at near C speeds

- Plays nicely with other languages $\Longrightarrow$ 0-overhead C ffi (without wrappers); call Python, R, Java, Matlab, ect. directly

### Unique features

- Novel approach to types and metaprogramming $\Longrightarrow$ allows you to write general algorithms, while generating specialized code for each combination datatypes

- Gives access to multiple intermediate representations of code on the way to compilation/execution
    - After parsing (`@code_lowered`)
    - After inference (`@code_typed`)
    - llvm IR (`@code_llvm`)
    - Native machine code (`@code_native`)

#### Example: benchmarking

- [Issue](https://github.com/johnmyleswhite/Benchmarks.jl/issues/5#issuecomment-129918554) I had when benchmarking what I thought were _very_ fast functions:
- Demonstration:

In [3]:
using Benchmarks
@benchmark sin(2.0)

================ Benchmark Results ========================
   Average elapsed time: 12.94 ns
     95% CI for average: [12.74 ns, 13.14 ns]
   Minimum elapsed time: 12.44 ns
                GC time: 0.00%
       Memory allocated: 0 bytes
  Number of allocations: 0 allocations
      Number of samples: 7501
        R² of OLS model: 0.952
Time used for benchmark: 0.10s
            Precompiled: true
       Multiple samples: true
       Search performed: true

In [4]:
# do f and y have the same sign?
_f2(x, y) = x*y > 0.0
_f2(1, 2.0)  # run once to compile

true

In [5]:
@benchmark _f2(42, 3.14159)

LoadError: LoadError: InexactError()
while loading In[5], in expression starting on line 28

- What's the problem here? 
- Here `_f2` reached `8467170729913085952` (see [GitHub issue]((https://github.com/johnmyleswhite/Benchmarks.jl/issues/5#issuecomment-129918554)) evaluations per sample before throwing an error. 
- Next geometric expansion overflowed the 64 bit integer:

In [18]:
# use `big` for arbitrary precision
typemax(Int64) < ceil(BigInt, 1.1 * big(8467170729913085952))

true

- How could Julia execute `_f2` 8 quintillion 467 quadrillion 170 trillion 729 billion 913 million 85 thousand 952  (8 billion billion) times in under 10 seconds?
- It can't:

In [7]:
function foo()
   out = false
   for i=1:8467170729913085952
        out = _f2(42, 3.14159)
   end
    return out
end  

foo (generic function with 1 method)

In [9]:
foo()

true

In [10]:
@benchmark foo()

================ Benchmark Results ========================
   Average elapsed time: 2.71 ns
     95% CI for average: [2.66 ns, 2.76 ns]
   Minimum elapsed time: 3.38 ns
                GC time: 0.00%
       Memory allocated: 0 bytes
  Number of allocations: 0 allocations
      Number of samples: 5601
        R² of OLS model: 0.952
Time used for benchmark: 0.03s
            Precompiled: true
       Multiple samples: true
       Search performed: true

- We can look at the intermediate representations of the function call to explore what is happening

In [11]:
# AST before inference -- not very enlightening
@code_lowered foo()

1-element Array{Any,1}:
 :($(Expr(:lambda, Any[], Any[Any[Any[:out,:Any,2],Any[symbol("#s259"),:Any,2],Any[:i,:Any,18]],Any[],2,Any[]], :(begin  # In[7], line 2:
        out = false # line 3:
        GenSym(0) = (Main.colon)(1,8467170729913085952)
        #s259 = (top(start))(GenSym(0))
        unless (top(!))((top(done))(GenSym(0),#s259)) goto 1
        2: 
        GenSym(1) = (top(next))(GenSym(0),#s259)
        i = (top(getfield))(GenSym(1),1)
        #s259 = (top(getfield))(GenSym(1),2) # line 4:
        out = (Main._f2)(42,3.14159)
        3: 
        unless (top(!))((top(!))((top(done))(GenSym(0),#s259))) goto 2
        1: 
        0:  # line 6:
        return out
    end))))

In [22]:
# AST after inference -- also not enlightening
@code_typed foo()

1-element Array{Any,1}:
 :($(Expr(:lambda, Any[], Any[Any[Any[:out,Bool,2],Any[symbol("#s872"),Int64,2],Any[:i,Int64,18]],Any[],Any[UnitRange{Int64},Tuple{Int64,Int64},Int64,Int64],Any[]], :(begin  # In[19], line 2:
        out = false # line 3:
        GenSym(0) = $(Expr(:new, UnitRange{Int64}, 1, :(((top(getfield))(Base.Intrinsics,:select_value)::I)((Base.sle_int)(1,8467170729913085952)::Bool,8467170729913085952,(Base.box)(Int64,(Base.sub_int)(1,1))::Int64)::Int64)))
        #s872 = (top(getfield))(GenSym(0),:start)::Int64
        unless (Base.box)(Base.Bool,(Base.not_int)(#s872::Int64 === (Base.box)(Base.Int,(Base.add_int)((top(getfield))(GenSym(0),:stop)::Int64,1))::Int64::Bool))::Bool goto 1
        2: 
        GenSym(2) = #s872::Int64
        GenSym(3) = (Base.box)(Base.Int,(Base.add_int)(#s872::Int64,1))::Int64
        i = GenSym(2)
        #s872 = GenSym(3) # line 4:
        out = (Base.lt_float)(0.0,(Base.box)(Base.Float64,(Base.mul_float)((Base.box)(Float64,(Base.sitofp)(Floa

In [24]:
# LLVM IR -- ahh, there's the problem
@code_llvm foo()


define i1 @julia_foo_24363() {
top:
  ret i1 true
}


In [12]:
@code_native foo()

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[7]
Source line: 6
	pushq	%rbp
	movq	%rsp, %rbp
	movb	$1, %al
Source line: 6
	popq	%rbp
	ret


## Example: `@generated` functions
- Task: Construct basis matrix for complete polynomial of degree `d`, given input data `z`. 
    - `z` is assumed to be the degree 1 realization(s) of each variable. 
    - For example, if variables are `q`, `r`, and `s`; then  
```julia
z = [q r s]
```


- Output is a basis matrix. 
- In our example, with `d` set to 2 we want a matrix with the following columns

$$\begin{bmatrix}\mathbf{1} & q & r & s & q r & q s & r s & q^2 & r^2 & s^2  \end{bmatrix}$$
- Notice no $q^2 s$, $q^2 r$, $q s^2$, ect. terms
- Efficiency boost is magnified as either `d` or the number of variables increases

#### Implementation

- One function per `d`

```julia
complete_monomial_1d(z::Matrix) = hcat(ones(size(z, 1), z)

function complete_monomial_2d(z::Matrix)
    n_x, n = size(z)
    out = Array(Float64, n_x, n_complete(n, 2))

    out[:, 1] = 1.0
    out[:, 2:n+1] = z
    
    i = n+1
    
    for j1=1:n
        for j2=j1:n
            i += 1
            out[:, i] = z[:, j1] .* z[:, j2]
        end
    end
    out
end

function complete_monomial_3d(z::Matrix)
    n_x, n = size(z)
    out = Array(Float64, n_x, n_complete(n, 3))

    out[:, 1] = 1.0
    out[:, 2:n+1] = z
    
    i = n+1
    
    for j1=1:n
        for j2=j1:n
            i += 1
            out[:, i] = z[:, j1] .* z[:, j2]
            for j3=j2:n
                i += 1
                out[:, i] = z[:, j1] .* z[:, j2] .* z[:, j3]
            end
        end
    end
    out
end

#...
```

- One function, manually checking `d`

```julia
function complete_monomial(z::Matrix, d::Int)
    if d > 4
        throw(ArgumentError("Can't handle more than d = 4"))
    end
        
    n_x, n = size(z)
    out = Array(Float64, n_x, n_complete(n, d))

    out[:, 1] = 1.0
    out[:, 2:n+1] = z

    i = n+1
    
    if d == 2
        for j1=1:n
            for j2=j1:n
                i += 1
                out[:, i] = z[:, j1] .* z[:, j2]
            end
        end
        
    elseif d == 3

        for j1=1:n
            for j2=j1:n
                i += 1
                out[:, i] = z[:, j1] .* z[:, j2]
                
                for j3=j2:n
                    out += 1
                    out[:, i] = z[:, j1] .* z[:, j2] .* z[:, j3]
                end
            end
        end    
        
    elseif d == 4
    
        for j1=1:n
            for j2=j1:n
                i += 1
                out[:, i] = z[:, j1] .* z[:, j2]
    
                for j3=j2:n
                    i += 1
                    out[:, i] = z[:, j1] .* z[:, j2] .* z[:, j3]
                
                    for j4=j3:n
                        i += 1
                        out[:, i] = z[:, j1] .* z[:, j2] .* z[:, j3] .* z[:, j4]
                    end
                end
            end
        end 
    
    
    return out
end
```

- That code is terrible and repetitive
- Solution: use `@generated` functions to get the compiler to write the code for us
- Functions can be `@generated` in magic time between type inference and compilation
- Allows the programmer to operate on **run-time** *types* of objects and generate specialized code

In [13]:
using Base.Cartesian  # load extra built-in metaprogramming tools

# type that will allow us to operate on arbitrary degree
immutable Degree{D} end

"""
Give the number of terms in the complete polynomial of degree 
`D` in `n` variables
"""
function n_complete(n::Int, D::Int)
    out = 1
    for d=1:D
        tmp = 1
        for j=0:d-1
            tmp *= (n+j)
        end
        out += div(tmp, factorial(d))
    end
    out
end

n_complete (generic function with 1 method)

In [41]:
"""
Our main function. Operates on the run-time _types_
that specify the element type of z and the degree of the polynomial.

Returns a Julia expression that can be expanded at compile time
into efficient, specialized general code.
"""
function complete_polynomial_impl!{T,D}(z::Type{Matrix{T}}, ::Type{Degree{D}},
                                        ::Type{Matrix{T}})
    println("I've never seen a $D before!!")
    quote
        nobs, nvar = size(z)
        if size(out) != (nobs, n_complete(nvar, $D))
            error("z, out not compatible")
        end

        # reset first column to ones
        @inbounds for i=1:nobs
            out[i, 1] = 1.0
        end
        
        # set next nvar columns to input matrix
        @inbounds for n=2:nvar+1, i=1:nobs
            out[i, n] = z[i, n-1]
        end
        
        
        ix = nvar+1
        @nloops($D, # number of loops
                i,  # counter
                d->((d == $D ? 1 : i_{d+1}) : nvar),  # ranges
                d->(d == $D ? nothing :
                    (begin
                        ix += 1
                        for r=1:nobs
                            tmp = one($T)
                            @nexprs $D-d+1 j->(tmp *= z[r, i_{$D-j+1}])
                            out[r, ix]=tmp
                        end
                    end)),  # preexpr
                Expr(:block, :nothing)  # bodyexpr
                )

        out
    end
end


# API so users don't have to think about how this happens...
@generated function complete_polynomial!{T,D}(z::Matrix{T}, d::Degree{D},
                                              out::Matrix{T})
    complete_polynomial_impl!(z, d, out)
end

function complete_polynomial{T}(z::Matrix{T}, d::Int)
    nobs, nvar = size(z)
    out = Array(T, nobs, n_complete(nvar, d))
    complete_polynomial!(z, Degree{d}(), out)::Matrix{T}
end

# function complete_polynomial!{T}(z::Matrix{T}, d::Int, out::Matrix{T})
#     complete_polynomial!(z, Degree{d}(), out)::Matrix{T}
# end


complete_polynomial (generic function with 1 method)

In [28]:
bing(x::Array{Int8, 3}, y::Array{Int8, 3}) = "hello"
bing(x::Array{Float64, 3}, y::Array{Float64, 3}) = " world!"
bing(x::Array, y::Array) = "π"
bing(x) = "holy crap"
bing(x, y...) = "holy holy crap"

bing (generic function with 6 methods)

In [36]:
bing(true, false)

"holy holy crap"

In [48]:
z = rand(Int64, 10, 3)
complete_polynomial(z, 3)

10x20 Array{Int64,2}:
 1   4090761276323375628   6174006380292642962  …   6169913072611006696
 1   1358796917882287569   3545568378408000695      6932434956123726600
 1  -8541259174508298418   6381260643360795474     -1847586742164302920
 1  -2986378239988943492   3031267148602184624     -1240785835727468191
 1   3873996157182141613  -5751673107759527698     -1025760715484026739
 1  -3375980518437612579  -3735941091538372018  …   3750448031811323464
 1     -6062322050398711  -8220903493712129751     -9110272432023281064
 1   3157306623872132589   -643559529426315433     -5271758905022099879
 1   6407705964348392006   -821188481491160567     -2385542082729961592
 1   2473456209025917346   1344953301422291173      -271967206275082183

In [21]:
z

10x3 Array{Float64,2}:
 0.439844   0.93496   0.429994 
 0.325276   0.382143  0.697281 
 0.858268   0.870925  0.182534 
 0.78834    0.896466  0.887134 
 0.761294   0.684034  0.436334 
 0.718588   0.240516  0.966357 
 0.0528628  0.190845  0.967564 
 0.151911   0.802862  0.26659  
 0.539701   0.149782  0.0786654
 0.614315   0.791584  0.414551 

In [17]:
macroexpand(complete_polynomial_impl!(Matrix{Float64}, Degree{8}, Matrix{Float64}))

quote  # In[14], line 11:
    (nobs,nvar) = size(z) # line 12:
    if size(out) != (nobs,n_complete(nvar,8)) # line 13:
        error("z, out not compatible")
    end # line 17:
    begin 
        $(Expr(:boundscheck, false))
        begin 
            for i = 1:nobs # line 18:
                out[i,1] = 1.0
            end
            $(Expr(:boundscheck, :(Base.pop)))
        end
    end # line 22:
    begin 
        $(Expr(:boundscheck, false))
        begin 
            for n = 2:nvar + 1, i = 1:nobs # line 23:
                out[i,n] = z[i,n - 1]
            end
            $(Expr(:boundscheck, :(Base.pop)))
        end
    end # line 27:
    ix = nvar + 1 # line 28:
    begin  # cartesian.jl, line 31:
        for i_8 = 1:nvar # line 32:
            nothing # line 33:
            begin  # cartesian.jl, line 31:
                for i_7 = i_8:nvar # line 32:
                    begin  # In[14], line 33:
                        ix += 1 # line 34:
                        for r = 1:no

Move to [Images](Images.ipynb) slides (time permitting)